In [ ]:
#Netid : aadcosta
#SUID : 594303899
#Pin No : 79

# Class function called Func is the utility functions containing the logic for different approaches and the installation of the required libraries for the same are done below

In [1]:
pip install fancyimpute

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\Arlene\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [4]:
import pandas as pd
import numpy as np
import configparser
from fancyimpute import IterativeImputer
import sys,os

Below class Func contains the functions for all different algorithms such as Mean,Median, Local Gradient, Nearest Neighbour(Both distance between data points using difference and Euclidean distance)

In [2]:
class Func:

    def format_to_two_decimal_places(self,value):
        return '{:.2f}'.format(value)


    def get_df(self,path):
        data_df = pd.read_csv(rf'{path}')
        selected_columns_df = data_df[['2017', '2018', '2019', '2020', '2021', '2022']]
        return selected_columns_df

    def get_nan_value(self,originaldf):
        nanno = []

        for i in range(originaldf.shape[0]):
            for j in range(originaldf.shape[1]):
                if np.isnan(originaldf.iloc[i, j]):
                    nanno.append([i, j])
        return nanno

    def get_nan_value_field(self,originaldf):
        '''Function for computing the index of the NaN values in the original dataframe for computing avg absolute Error'''
        nanno = []

        for j in range(originaldf.shape[1]):
            for i in range(originaldf.shape[0]):
                if np.isnan(originaldf.iloc[i, j]):
                    nanno.append([i, j])
        print(nanno)
        return nanno

    
    def get_value(self,nanarray,df):
        '''Get NaN valued elements within dataframe for computing absolute Error between original and missing value dataframe '''
        arrval = []
        for val in nanarray:
            arrval.append(df.iloc[val[0], val[1]])
        return arrval

    def get_absolute_error(self,actual_values,imputed_values):
        '''Function for computing avg absolute Error'''
        absolute_errors = []
        for i in range(0,len(actual_values)):
            absolute_errors.append(abs(actual_values[i] - imputed_values[i]))


        # Calculate the average absolute error
        average_absolute_error = sum(absolute_errors)//len(absolute_errors)

        return average_absolute_error

    def replace_nan_with_mean(self,row):
        '''Function for computing mean for non-nan valued locations[Can be either row-wise or col wise based on axis]'''
        mean = row.mean(skipna=True)  # Calculate row mean excluding NaN values
        return row.fillna(mean)  # Replace NaN with the mean

    def replace_nan_with_median(self,row):
        '''Function for computing median for non-nan valued locations[Can be either row-wise or col wise based on axis]'''
        median = row.median(skipna=True)  # Calculate row mean excluding NaN values
        return row.fillna(median)  # Replace NaN with the mean


    def get_local_gradient(self,df):
        '''Function for capturing the gradient based on nearest values'''
        for i in range(df.shape[0]):
            for j in range(df.shape[1]):
                if np.isnan(df.iloc[i, j]):
                    if j == 0:
                        df.iloc[i, j] = 2 * df.iloc[i, 1] - df.iloc[i, 2]
                    elif j == 5:
                        df.iloc[i, j] = 2 * df.iloc[i, 4] - df.iloc[i, 3]
                    else:

                        preceding_year = j - 1
                        following_year = j + 1

                        df.iloc[i, j] = (df.iloc[i, preceding_year] + df.iloc[i, following_year]) / 2
        return df


    # Define the L1 distance function
    def l1_distance(self,row1, row2):
        '''Formula for capturing distances between two rows excluding the NaN indexes'''
        common_fields = set(row1.dropna().index) & set(row2.dropna().index)
        distance = 0
        count = 0

        for field in common_fields:
            distance += abs(row1[field] - row2[field])
            count += 1

        return distance / count if count > 0 else float('inf')

        # Define the L2 distance function : Average Squared differences
    def l2_distance(self, row1, row2):
        '''Formula for capturing distance between two rows based on Euclidean distance'''
        common_fields = set(row1.dropna().index) & set(row2.dropna().index)
        distance = 0
        count = 0

        for field in common_fields:
            distance += (row1[field] - row2[field])**2
            count += 1

        return distance / count if count > 0 else float('inf')

    def get_nn(self,df,type):
        '''Function for computing nn algo by excluding rows having same NaN indexes'''
        dfres = df.copy(deep=True)
        for i, row1 in df.iterrows():
            min_distance = float('inf')  # Initialize the minimum distance to infinity
            nearest_neighbor = None  # Initialize the nearest neighbor

            for j, row2 in df.iterrows():
                if i != j:  # Exclude the current faculty member
                    # Check if the current faculty member has missing values in the same field as the neighbor
                    missing_fields_i = set(row1.index[row1.isna()])
                    missing_fields_j = set(row2.index[row2.isna()])
                    if not missing_fields_i & missing_fields_j:
                        if type == 'l1':
                            distance = self.l1_distance(row1, row2)
                        elif type == 'l2':
                            distance = self.l2_distance(row1, row2)
                        if distance < min_distance:
                            #print(type,distance,i,j)
                            min_distance = distance
                            nearest_neighbor = row2

            # Replace missing values in the current faculty member with values from the nearest neighbor
            if nearest_neighbor is not None:
                for field in row1.index[row1.isna()]:
                    dfres.at[i, field] = nearest_neighbor[field]
        return dfres


Q1 : General Imputation Algorithm( I have used Mice based on FancyImpute [Library available in Python])

In [5]:
original_path = 'E:\MS\MS_studies\Sem_2\IntrotoML\HW0\Missouri.csv'
missing_path = 'E:\MS\MS_studies\Sem_2\IntrotoML\HW0\missingNoisyMissouri.csv'

In [6]:
f = Func()
originaldf = f.get_df(rf'{original_path}')
print("Original df:")
print(originaldf)

# Step 2: Capture Missing CSV
missingdf = f.get_df(rf'{missing_path}')

# Step 3: Get the cell list of NaN values: 
nanlist = f.get_nan_value(missingdf)

# step 4: Impute values using imputation algorithms
mice_imputer = IterativeImputer()
# imputing the missing value with mice imputer
data = mice_imputer.fit_transform(missingdf)
missingdf = pd.DataFrame(data)
print("Missing df imputed with Mice is:")
print(missingdf)

# Step 5: Capture actual and pred values
actual_value = f.get_value(nanlist,originaldf)
pred_value = f.get_value(nanlist,missingdf)

# Step 6: Display the error
avg_abs_error = f.get_absolute_error(actual_value,pred_value)
print(f"Average Absolute Error using Mice is {avg_abs_error}")

Original df:
   2017  2018  2019  2020  2021  2022
0    13     5     8     8    12     9
1  2293  2249  2193  2218  2152  2094
2     4     2     3     7     4     3
3   336   360   337   321   298   289
4   313   309   256   310   306   279
5    39    48    33    36    43    38
6    14    20    23    69   114   107
7    23    44   150   182   255   268
8    34    44    60    59    38    67
9    28    34    34    16    33    33
Missing df imputed with Mice is:
            0            1            2            3            4            5
0    15.00000    25.188646     2.000000     3.000000    18.000000     4.000000
1  2298.00000  2239.000000  2054.436211  2208.000000  2154.000000  2098.000000
2     9.00000    18.644771     1.000000    10.000000     2.000000    13.000000
3   338.00000   360.000000   337.000000   307.680052   298.000000   289.000000
4   323.00000   314.000000   250.000000   305.000000   302.000000   293.045661
5    40.00000    43.000000    37.000000    34.776230    44.000

Q2: Mean[Sum of all non nan data/len(row)] and Field mean[Sum of all non Nan data(col wise)/len(col)]
Code for the same is written in the class Func and function name is replace_nan_with_mean

In [7]:
originaldf = f.get_df(rf'{original_path}')


# Step 2: Capture Missing CSV
missingdf = f.get_df(rf'{missing_path}')

# Step 3: Get the cell list of NaN values
nanlist = f.get_nan_value(missingdf)

# step 4: Find mean
missingdf_mean = missingdf.apply(f.replace_nan_with_mean, axis=1)
print("Meandf:")
print(missingdf)
# Step 5: Capture actual and pred values
actual_value = f.get_value(nanlist,originaldf)
pred_value = f.get_value(nanlist,missingdf_mean)

# Step 6: Display the error
avg_abs_error = f.get_absolute_error(actual_value,pred_value)
print(f"Average Absolute Error using Mean is {avg_abs_error}")

# step 7: Find mean
missingdf_fmean = missingdf.apply(f.replace_nan_with_mean, axis=0)
print("Field Mean df")
print(missingdf_fmean)
# Step 8: Capture actual and pred values
actual_value_new = f.get_value(nanlist,originaldf)
pred_value_new = f.get_value(nanlist,missingdf_fmean)
#(actual_value_new,pred_value_new)
# Step 9: Display the error
avg_abs_error = f.get_absolute_error(actual_value_new,pred_value_new)
print(f"Average Absolute Error using Field Mean is {avg_abs_error}")

Meandf:
     2017    2018   2019    2020    2021    2022
0    15.0     NaN    2.0     3.0    18.0     4.0
1  2298.0  2239.0    NaN  2208.0  2154.0  2098.0
2     9.0     NaN    1.0    10.0     2.0    13.0
3   338.0   360.0  337.0     NaN   298.0   289.0
4   323.0   314.0  250.0   305.0   302.0     NaN
5    40.0    43.0   37.0     NaN    44.0    39.0
6    15.0    22.0   28.0    70.0     NaN   114.0
7     NaN    35.0  142.0   189.0   252.0   266.0
8    32.0    40.0   64.0     NaN    34.0    60.0
9    20.0     NaN   35.0    17.0    30.0    32.0
Average Absolute Error using Mean is 28.0
Field Mean df
          2017         2018        2019         2020         2021         2022
0    15.000000   436.142857    2.000000     3.000000    18.000000     4.000000
1  2298.000000  2239.000000   99.555556  2208.000000  2154.000000  2098.000000
2     9.000000   436.142857    1.000000    10.000000     2.000000    13.000000
3   338.000000   360.000000  337.000000   400.285714   298.000000   289.000000
4 

Median and Field Median
Code for the same is written in the class Func and function name is replace_nan_with_median


In [8]:
# Step 3: Get the cell list of NaN values as Step 1 and 2 will be same 
nanlist = f.get_nan_value(missingdf)


# step 4: Find mean
missingdf_med = missingdf.apply(f.replace_nan_with_median, axis=1)
print("Mediandf")
print(missingdf_med)
# Step 5: Capture actual and pred values
actual_value = f.get_value(nanlist,originaldf)
pred_value = f.get_value(nanlist,missingdf_med)

# Step 6: Display the error
avg_abs_error = f.get_absolute_error(actual_value,pred_value)
print(f"Average Absolute Error using Median is {avg_abs_error}")

# step 7: Find field mean
missingdf_fmed = missingdf.apply(f.replace_nan_with_median, axis=0)
print("Field Median df")
print(missingdf_fmed)

# Step 8: Capture actual and pred values
actual_value_field = f.get_value(nanlist,originaldf)
pred_value_field = f.get_value(nanlist,missingdf_fmed)

# Step 9: Display the error for Field Median
avg_abs_error = f.get_absolute_error(actual_value_field,pred_value_field)
print(f"Average Absolute Error using Field Median is {avg_abs_error}")

Mediandf
     2017    2018    2019    2020    2021    2022
0    15.0     4.0     2.0     3.0    18.0     4.0
1  2298.0  2239.0  2208.0  2208.0  2154.0  2098.0
2     9.0     9.0     1.0    10.0     2.0    13.0
3   338.0   360.0   337.0   337.0   298.0   289.0
4   323.0   314.0   250.0   305.0   302.0   305.0
5    40.0    43.0    37.0    40.0    44.0    39.0
6    15.0    22.0    28.0    70.0    28.0   114.0
7   189.0    35.0   142.0   189.0   252.0   266.0
8    32.0    40.0    64.0    40.0    34.0    60.0
9    20.0    30.0    35.0    17.0    30.0    32.0
Average Absolute Error using Median is 34.0
Field Median df
     2017    2018   2019    2020    2021    2022
0    15.0    43.0    2.0     3.0    18.0     4.0
1  2298.0  2239.0   37.0  2208.0  2154.0  2098.0
2     9.0    43.0    1.0    10.0     2.0    13.0
3   338.0   360.0  337.0    70.0   298.0   289.0
4   323.0   314.0  250.0   305.0   302.0    60.0
5    40.0    43.0   37.0    70.0    44.0    39.0
6    15.0    22.0   28.0    70.0    44

Local Gradient obtained by averaging on the preceding and succeeding elements for the faculty member 
Code for the same is written in the class Func and function name is get_local_gradient


In [9]:
# Step 3: Get the cell list of NaN values
nanlist = f.get_nan_value(missingdf)

# step 4: Impute values using LocalGradient

missingdfgrad = f.get_local_gradient(missingdf)
print("LocalGradientdf")
print(missingdfgrad)

# Step 5: Capture actual and pred values
actual_value = f.get_value(nanlist,originaldf)
pred_value = f.get_value(nanlist,missingdfgrad)

# Step 6: Display the error
avg_abs_error = f.get_absolute_error(actual_value,pred_value)
print(f"Average Absolute Error using LocalGradient is {avg_abs_error}")

LocalGradientdf
     2017    2018    2019    2020    2021    2022
0    15.0     8.5     2.0     3.0    18.0     4.0
1  2298.0  2239.0  2223.5  2208.0  2154.0  2098.0
2     9.0     5.0     1.0    10.0     2.0    13.0
3   338.0   360.0   337.0   317.5   298.0   289.0
4   323.0   314.0   250.0   305.0   302.0   299.0
5    40.0    43.0    37.0    40.5    44.0    39.0
6    15.0    22.0    28.0    70.0    92.0   114.0
7   -72.0    35.0   142.0   189.0   252.0   266.0
8    32.0    40.0    64.0    49.0    34.0    60.0
9    20.0    27.5    35.0    17.0    30.0    32.0
Average Absolute Error using LocalGradient is 19.0


Nearest Neighbour(L1[distance based on difference between Data points] and L2[Euclidean Distance])
Code for the same is written in the class Func and function name is get_nn

In [10]:
originaldf = f.get_df(rf'{original_path}')
#print(originaldf)

# Step 2: Capture Missing CSV
missingdf = f.get_df(rf'{missing_path}')

# Step 3: Get the cell list of NaN values
nanlist = f.get_nan_value(missingdf)

# step 4: Impute values using NN1

missingdfl1 = f.get_nn(missingdf,"l1")
print("Nearest Neighbour based on data point difference:")
print(missingdfl1)
# Step 5: Capture actual and pred values
actual_value = f.get_value(nanlist,originaldf)
pred_value = f.get_value(nanlist,missingdfl1)

# Step 6: Display the error
avg_abs_error = f.get_absolute_error(actual_value,pred_value)
print(f"Average Absolute Error using NearestNeighbourL1 is {avg_abs_error}")

# step 7: Impute values using NN2

missingdfl2 = f.get_nn(missingdf,"l2")
print("Nearest Neighbour based on Euclidean data point difference:")
print(missingdfl2)
# Step 8: Capture actual and pred values
actual_value = f.get_value(nanlist,originaldf)
pred_value = f.get_value(nanlist,missingdfl2)

# Step 9: Display the error
avg_abs_error = f.get_absolute_error(actual_value,pred_value)
print(f"Average Absolute Error using NearestNeighbourL2 is {avg_abs_error}")

Nearest Neighbour based on data point difference:
     2017    2018   2019    2020    2021    2022
0    15.0    43.0    2.0     3.0    18.0     4.0
1  2298.0  2239.0  337.0  2208.0  2154.0  2098.0
2     9.0    43.0    1.0    10.0     2.0    13.0
3   338.0   360.0  337.0   305.0   298.0   289.0
4   323.0   314.0  250.0   305.0   302.0   289.0
5    40.0    43.0   37.0    17.0    44.0    39.0
6    15.0    22.0   28.0    70.0    34.0   114.0
7    15.0    35.0  142.0   189.0   252.0   266.0
8    32.0    40.0   64.0    17.0    34.0    60.0
9    20.0    43.0   35.0    17.0    30.0    32.0
Average Absolute Error using NearestNeighbourL1 is 211.0
Nearest Neighbour based on Euclidean data point difference:
     2017    2018   2019    2020    2021    2022
0    15.0    43.0    2.0     3.0    18.0     4.0
1  2298.0  2239.0  337.0  2208.0  2154.0  2098.0
2     9.0    43.0    1.0    10.0     2.0    13.0
3   338.0   360.0  337.0   305.0   298.0   289.0
4   323.0   314.0  250.0   305.0   302.0   289.0


In [ ]:
Q3] Comparison of different approaches:
---------------------------------------------------------

From the above code comparison of Average Absolute Error is as follows :
	Mice	Mean	FieldMean	Median	FieldMedian	LocalGradient	Nearest Neighbour L1	Nearest Neighbour L2
    26		28		474         34			283      19             211                     211

    Local Gradient provides the least Average Absolute Error as noise was added to the citations distributed yearwise for each faculty member, by 
    obtaining local Gradient based on averaging the previous and following year citations value, the average obtained is closer to the real value.
    The csv also follows a specific pattern of adding noise based on +10 or -10  and hence can also be approximated well by local relationships.

    Generally, median performs better on outliers, however we can see that data is skewed towards the right, hence the median is greater than mean.

    For FieldMean , the error is maximum as citations might vary for every faculty member yearwise as there is no relationship/pattern between the year 
    and the faculty member and computing the average on different varying values might lead to incorrect imputated values leading to incorrect results.

    Nearest Neighbour doesnt reduce the error as although the distance might be lesser between the citations for the faculty member,
    however the citation might vary for each faculty member.